#### smote + enn
          precision    recall  f1-score   support

          0       0.98      0.96      0.97      1920
          1       0.35      0.51      0.41        80

avg / total       0.95      0.94      0.95      2000

ACCURACY: 0.942

#### smote enn 결측치 2개 delete 했을 때

          precision    recall  f1-score   support

          0       0.98      0.95      0.96      1920
          1       0.28      0.51      0.37        80

avg / total       0.95      0.93      0.94      2000

ACCURACY: 0.929

#### ADASYN

          precision    recall  f1-score   support

          0       0.98      0.97      0.97      1920
          1       0.38      0.46      0.42        80

avg / total       0.95      0.95      0.95      2000

ACCURACY: 0.9485


#### Random oversampling
             precision    recall  f1-score   support

          0       0.98      0.94      0.96      1920
          1       0.30      0.65      0.41        80

avg / total       0.96      0.93      0.94      2000

ACCURACY: 0.9265


### 비식별처리 replace , fillna 해도 비식별처리로 인식됨

In [2]:
import pandas as pd
import numpy as np
import os
# from imblearn.combine import SMOTEENN

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import classification_report
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,normalize
from sklearn import preprocessing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df = pd.read_csv('../../data/Data_set.csv',encoding='cp949')
df1 = pd.read_csv('../../data/Test_set.csv',encoding='cp949')
df1.AGE.value_counts()

50    382
45    374
55    322
40    309
35    214
60    191
30     92
65     68
25     53
70      5
20      5
*       4
Name: AGE, dtype: int64

In [3]:
# df1.SEX.value_counts(0)
# df.TARGET.value_counts()


In [4]:
# def plot_correlation_map( df ):
#     corr = df.corr()
#     _ , ax = plt.subplots( figsize =( 48 , 40 ) )
#     cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
#     _ = sns.heatmap(
#         corr, 
#         cmap = cmap,
#         square=True, 
#         cbar_kws={ 'shrink' : .9 }, 
#         ax=ax, 
#         annot = True, 
#         annot_kws = { 'fontsize' : 15 }
#     )

In [5]:
# plot_correlation_map( df )

In [6]:
# df.info()

In [7]:
df.isnull().sum()

CUST_ID                  0
TARGET                   0
BNK_LNIF_CNT             0
CPT_LNIF_CNT             0
SPART_LNIF_CNT           0
ECT_LNIF_CNT             0
TOT_LNIF_AMT             0
TOT_CLIF_AMT             0
BNK_LNIF_AMT             0
CPT_LNIF_AMT             0
CRDT_OCCR_MDIF           0
SPTCT_OCCR_MDIF          0
CRDT_CARD_CNT            0
CTCD_OCCR_MDIF           0
CB_GUIF_CNT              0
CB_GUIF_AMT              0
OCCP_NAME_G            464
CUST_JOB_INCM            0
HSHD_INFR_INCM           0
ACTL_FMLY_NUM            0
CUST_FMLY_NUM            0
LAST_CHLD_AGE         1027
MATE_OCCP_NAME_G     45709
MATE_JOB_INCM            0
CRDT_LOAN_CNT            0
MIN_CNTT_DATE            0
TOT_CRLN_AMT             0
TOT_REPY_AMT             0
CRLN_OVDU_RATE           0
CRLN_30OVDU_RATE         0
                     ...  
GDINS_MON_PREM           0
SVINS_MON_PREM           0
FMLY_GDINS_MNPREM        0
FMLY_SVINS_MNPREM        0
MAX_MON_PREM             0
TOT_PREM                 0
F

## MATE_OCCP_NAME_G * 제거

In [15]:
#주부의 추정 소득이 0이고 NaN의 추정소득이 0이기에 NaN값을 주부로 바꿔줌
#고소득 전문직 5848.75
df.MATE_OCCP_NAME_G = df["MATE_OCCP_NAME_G"].fillna("주부")
print(df.MATE_OCCP_NAME_G.value_counts())
of_index = df.MATE_OCCP_NAME_G[df.MATE_OCCP_NAME_G=="2차산업 종사자"].index.tolist()
df.MATE_JOB_INCM.loc[of_index].mean()

주부          57808
사무직         10053
2차산업 종사자     8320
자영업          6380
3차산업 종사자     4011
운전직          2887
공무원          2400
전문직          2105
1차산업 종사자     1411
단순 사무직       1316
고소득 전문직       958
기업/단체 임원      931
단순 노무직        549
기타            534
예체능계 종사자      461
학생            109
Name: MATE_OCCP_NAME_G, dtype: int64


3813.7620192307691

In [13]:
star_index = df[df['MATE_OCCP_NAME_G'] == "*"].index.tolist()

In [14]:
#직업이 *인 사람 중 추정 소득이 0인 사람 주부로 바꿔줌
star_data = df.MATE_JOB_INCM.loc[star_index]
zero_index = star_data[star_data==0].index.tolist()
df.MATE_OCCP_NAME_G.loc[zero_index] = "주부"
no_zero = star_data[star_data != 0]
#* 추정소득 4500이상인 사람 고소득 전문직으로 변경 
#* 추정소득 4500미만인 사람 
high_index = no_zero[no_zero >= 4500].index.tolist()
low_index = no_zero[no_zero < 4500].index.tolist()
df.MATE_OCCP_NAME_G.loc[high_index] = "고소득 전문직"
df.MATE_OCCP_NAME_G.loc[low_index] = "2차산업 종사자"


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## OCCP_NAME_G *제거

In [17]:
print(df.OCCP_NAME_G.value_counts())
df.OCCP_NAME_G = df["OCCP_NAME_G"].fillna("주부")
s_index = df[df["OCCP_NAME_G"]=="*"].index.tolist()
s_data = df.CUST_JOB_INCM.loc[s_index]
zero_ix = s_data[s_data==0].index.tolist()
df.OCCP_NAME_G[zero_ix] = "주부"
#고소득 전문직, 단순 사무직
on_index = df.OCCP_NAME_G[df.OCCP_NAME_G=="단순 사무직"].index.tolist()
df.CUST_JOB_INCM.loc[on_index].mean()
h_index = s_data[s_data >= 4700].index.tolist()
l_index = s_data[s_data < 4700].index.tolist()
df.OCCP_NAME_G.loc[h_index] = "고소득 전문직"
df.OCCP_NAME_G.loc[l_index] = "단순 사무직"

주부          28029
사무직         16581
2차산업 종사자     9601
자영업          9485
3차산업 종사자     8275
공무원          5091
전문직          5043
단순 사무직       4889
학생           3835
운전직          2126
기타           1672
고소득 전문직      1630
1차산업 종사자     1178
기업/단체 임원     1041
예체능계 종사자      936
단순 노무직        821
Name: OCCP_NAME_G, dtype: int64


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
df.AGE

df.AGE = df.AGE.replace('*',46)
df.AGE = pd.Series(df.AGE).convert_objects(convert_numeric=True)
df.AGE = df.AGE.replace(0,df.AGE.mean())
df.AGE.value_counts()


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


45    19052
50    17734
55    15321
40    14807
35    12003
60     9190
30     5268
65     3584
25     2201
46      430
70      403
20      240
Name: AGE, dtype: int64

In [19]:
df.SEX
df.SEX = df.SEX.replace('*',1)
df.SEX = pd.Series(df.SEX).convert_objects(convert_numeric=True)
df.SEX.value_counts()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until


2    54490
1    45743
Name: SEX, dtype: int64

In [33]:
print(df.columns)
n_index = df.LAST_CHLD_AGE.loc[df.LAST_CHLD_AGE.isnull()].index.tolist()
df.AGE.loc[n_index].value_counts()
age_index = df.AGE[df["AGE"]==45].index.tolist()
print(df["AGE"].value_counts())
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 34].mean() #10.410660008684324
c_index = not_chld[not_chld >= 34] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 10.410660008684324

Index(['CUST_ID', 'TARGET', 'BNK_LNIF_CNT', 'CPT_LNIF_CNT', 'SPART_LNIF_CNT',
       'ECT_LNIF_CNT', 'TOT_LNIF_AMT', 'TOT_CLIF_AMT', 'BNK_LNIF_AMT',
       'CPT_LNIF_AMT', 'CRDT_OCCR_MDIF', 'SPTCT_OCCR_MDIF', 'CRDT_CARD_CNT',
       'CTCD_OCCR_MDIF', 'CB_GUIF_CNT', 'CB_GUIF_AMT', 'OCCP_NAME_G',
       'CUST_JOB_INCM', 'HSHD_INFR_INCM', 'ACTL_FMLY_NUM', 'CUST_FMLY_NUM',
       'LAST_CHLD_AGE', 'MATE_OCCP_NAME_G', 'MATE_JOB_INCM', 'CRDT_LOAN_CNT',
       'MIN_CNTT_DATE', 'TOT_CRLN_AMT', 'TOT_REPY_AMT', 'CRLN_OVDU_RATE',
       'CRLN_30OVDU_RATE', 'LT1Y_CLOD_RATE', 'STRT_CRDT_GRAD',
       'LTST_CRDT_GRAD', 'PREM_OVDU_RATE', 'LT1Y_PEOD_RATE', 'AVG_STLN_RATE',
       'STLN_REMN_AMT', 'LT1Y_STLN_AMT', 'LT1Y_SLOD_RATE', 'GDINS_MON_PREM',
       'SVINS_MON_PREM', 'FMLY_GDINS_MNPREM', 'FMLY_SVINS_MNPREM',
       'MAX_MON_PREM', 'TOT_PREM', 'FMLY_TOT_PREM', 'CNTT_LAMT_CNT',
       'LT1Y_CTLT_CNT', 'AUTR_FAIL_MCNT', 'FYCM_PAID_AMT', 'FMLY_CLAM_CNT',
       'FMLY_PLPY_CNT', 'AGE', 'SEX', 'AVG_CAL

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
age_index = df.AGE[df["AGE"]==50].index.tolist()
data = df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 34].mean()#14.201807409180681
c_index = not_chld[not_chld >= 34] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 14.201807409180681

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [39]:
age_index = df.AGE[df["AGE"]==55].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 39].mean()#16.980453972257251
c_index = not_chld[not_chld >= 39] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 16.980453972257251

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [41]:
age_index = df.AGE[df["AGE"]==40].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 24].mean()#5.1291520979020975
c_index = not_chld[not_chld >= 24] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 5.1291520979020975

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [43]:
age_index = df.AGE[df["AGE"]==35].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 19].mean()# 1.7415281775136942
c_index = not_chld[not_chld >= 19] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 1.7415281775136942

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [45]:
age_index = df.AGE[df["AGE"]==60].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 44].mean()#19.987305905020968
c_index = not_chld[not_chld >= 44] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 19.987305905020968

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
age_index = df.AGE[df["AGE"]==30].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 14].mean()#0.097078651685393258
c_index = not_chld[not_chld >= 14] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 0.097078651685393258

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
age_index = df.AGE[df["AGE"]==65].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 49].mean()#21.332449160035367
c_index = not_chld[not_chld >= 49] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 21.332449160035367
df.LAST_CHLD_AGE.loc[age_index].value_counts()

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0.000000     1424
39.000000     891
44.000000     478
34.000000     273
21.332449     191
29.000000     113
24.000000      95
19.000000      56
14.000000      34
9.000000       21
4.000000        8
Name: LAST_CHLD_AGE, dtype: int64

In [51]:
age_index = df.AGE[df["AGE"]==25].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 9].mean()#0.056338028169014086
c_index = not_chld[not_chld >= 9] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 0.056338028169014086

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [53]:
age_index = df.AGE[df["AGE"]==70].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 54].mean()#22.603491271820449
c_index = not_chld[not_chld >= 54] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 22.603491271820449

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [55]:
age_index = df.AGE[df["AGE"]==20].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
c_index = not_chld[not_chld > 0] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 0

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [57]:
age_index = df.AGE[df["AGE"]==46].index.tolist()
df.LAST_CHLD_AGE.loc[age_index].value_counts()
not_chld = df.LAST_CHLD_AGE.loc[age_index]
not_chld[not_chld < 29].mean()#0.62937062937062938
c_index = not_chld[not_chld >= 29] .index.tolist()
df.LAST_CHLD_AGE.loc[c_index] = 0.62937062937062938

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [60]:
n_index = df.LAST_CHLD_AGE.loc[df.LAST_CHLD_AGE.isnull()].index.tolist()
null_data = df.AGE.loc[n_index]
four_six = null_data[null_data==46].index.tolist()
df.LAST_CHLD_AGE.loc[four_six] = 0.62937062937062938
five_five = null_data[null_data==55].index.tolist()
df.LAST_CHLD_AGE.loc[five_five] = 16.980453972257251
fourty = null_data[null_data==40].index.tolist()
df.LAST_CHLD_AGE.loc[fourty] = 5.1291520979020975
three_five = null_data[null_data==35].index.tolist()
df.LAST_CHLD_AGE.loc[three_five] = 1.7415281775136942
four_five = null_data[null_data==45].index.tolist()
df.LAST_CHLD_AGE.loc[four_five] = 10.410660008684324
fifty = null_data[null_data==50].index.tolist()
df.LAST_CHLD_AGE.loc[fifty] = 14.201807409180681
six_five = null_data[null_data==65].index.tolist()
df.LAST_CHLD_AGE.loc[six_five] = 21.332449160035367
sixty = null_data[null_data==60].index.tolist()
df.LAST_CHLD_AGE.loc[sixty] = 19.987305905020968

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
df.isnull().sum()


CUST_ID                  0
TARGET                   0
BNK_LNIF_CNT             0
CPT_LNIF_CNT             0
SPART_LNIF_CNT           0
ECT_LNIF_CNT             0
TOT_LNIF_AMT             0
TOT_CLIF_AMT             0
BNK_LNIF_AMT             0
CPT_LNIF_AMT             0
CRDT_OCCR_MDIF           0
SPTCT_OCCR_MDIF          0
CRDT_CARD_CNT            0
CTCD_OCCR_MDIF           0
CB_GUIF_CNT              0
CB_GUIF_AMT              0
OCCP_NAME_G              0
CUST_JOB_INCM            0
HSHD_INFR_INCM           0
ACTL_FMLY_NUM            0
CUST_FMLY_NUM            0
LAST_CHLD_AGE            0
MATE_OCCP_NAME_G         0
MATE_JOB_INCM            0
CRDT_LOAN_CNT            0
MIN_CNTT_DATE            0
TOT_CRLN_AMT             0
TOT_REPY_AMT             0
CRLN_OVDU_RATE           0
CRLN_30OVDU_RATE         0
                     ...  
GDINS_MON_PREM           0
SVINS_MON_PREM           0
FMLY_GDINS_MNPREM        0
FMLY_SVINS_MNPREM        0
MAX_MON_PREM             0
TOT_PREM                 0
F

In [61]:
df.TEL_CNTT_QTR = df.TEL_CNTT_QTR.astype(str)

In [62]:
# df.TEL_CNTT_QTR
# R>W>Q W1238050000  Q938800000
# print(df.TEL_MBSP_GRAD.value_counts())
# t_index = df.TEL_MBSP_GRAD.loc[df.TEL_MBSP_GRAD.isnull()].index.tolist()
# print(df.ARPU.loc[t_index].value_counts())#38917.309355644895
# print(df.ARPU[df.TEL_MBSP_GRAD=="R"].mean())#39648.85250348215
# r_index = df.ARPU[df.ARPU==0].index.tolist()
# df.OCCP_NAME_G.loc[r_index].value_counts()
# lindex = df.ARPU[df.ARPU==0].index.tolist()
# z_data = df.TEL_MBSP_GRAD.loc[lindex]
# qindex = z_data[z_data=="Q"].index.tolist()
# df.ARPU.loc[qindex] = 39648.85250348215
# windex = z_data[z_data=="W"].index.tolist()
# df.ARPU.loc[windex] = 41806.2541337
# eindex = z_data[z_data=="E"].index.tolist()
# df.ARPU.loc[eindex] = 63760.8744718
# eindex = z_data[z_data=="R"].index.tolist()
# df.ARPU.loc[eindex] = 55636.8943712
mindex = df.ARPU[df.ARPU==-1].index.tolist()
m_data = df.TEL_MBSP_GRAD.loc[mindex]
qindex = m_data[m_data=="Q"].index.tolist()
df.ARPU.loc[qindex] = 39648.85250348215
windex = m_data[m_data=="W"].index.tolist()
df.ARPU.loc[windex] = 41806.2541337
eindex = m_data[m_data=="E"].index.tolist()
df.ARPU.loc[eindex] = 63760.8744718
eindex = m_data[m_data=="R"].index.tolist()
df.ARPU.loc[eindex] = 55636.8943712

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [63]:
df.PAYM_METD = df.PAYM_METD.fillna('O')
df.TEL_MBSP_GRAD = df.TEL_MBSP_GRAD.fillna('Q')

In [64]:
del df['CUST_ID']
df = pd.get_dummies(df)

In [65]:
df_test = df[:2000]
df_train = df[2000:]

In [66]:
x_data = df_train.drop('TARGET',axis=1)
y_data = df_train.TARGET

In [67]:
X = x_data
y = y_data
from imblearn.over_sampling import ADASYN 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import CondensedNearestNeighbour 
from imblearn.combine import SMOTEENN 

# undersampling
# cnn = CondensedNearestNeighbour(random_state=42) 
# X_resampled, y_resampled = cnn.fit_sample(X, y)

#RANDOM 
# ros = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = ros.fit_sample(X, y)

#ADASYN
ada = ADASYN(random_state=42,n_neighbors=3)
X_resampled, y_resampled = ada.fit_sample(X, y)

# Apply SMOTE + ENN
# sm = SMOTEENN()
# X_resampled, y_resampled = sm.fit_sample(X, y)

In [68]:
df_1 = pd.DataFrame(X_resampled)
df_2 = pd.DataFrame(y_resampled)
df_2.columns = ['TARGET']
df_final = pd.concat([df_1,df_2],axis=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,202,TARGET
0,0.0,0.0,1.0,1.0,48001.0,48001.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
1,2.0,0.0,0.0,0.0,48001.0,1.0,48001.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
2,1.0,2.0,2.0,0.0,15001.0,3001.0,3001.0,12001.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
3,1.0,0.0,0.0,0.0,9001.0,9001.0,9001.0,0.0,25.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
4,2.0,0.0,0.0,0.0,39001.0,39001.0,39001.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0


In [69]:
df_final = df_final.sample(frac=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,202,TARGET
80308,1.000000,0.000000,0.000000,0.000000,72001.000000,72001.000000,72001.000000,0.000000,1.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,1.0,0
107612,1.000000,2.494517,2.494517,0.000000,28163.006937,28163.006937,18808.566347,9355.064220,1.000000,23.074287,...,0.0,0.0,0.0,0.000000,0.000000,0.376371,0.623629,0.0,1.0,1
135049,1.051115,0.948885,1.000000,0.051115,129787.264612,13074.416109,117480.574295,11387.568251,12.386619,14.226761,...,0.0,0.0,0.0,0.051115,0.948885,0.000000,0.000000,0.0,1.0,1
148779,0.617432,0.382568,1.000000,0.617432,7148.704988,5296.409976,1852.912444,3443.497532,84.273440,84.273440,...,0.0,0.0,0.0,0.000000,0.382568,0.617432,0.000000,0.0,1.0,1
28918,1.000000,0.000000,0.000000,0.000000,141001.000000,0.000000,141001.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,1.0,0


In [70]:
train_y = df_final.TARGET
train_x = df_final.drop('TARGET',axis=1)

In [71]:
test_y = df_test.TARGET
test_x = df_test.drop('TARGET',axis=1)

In [72]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,5))
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.fit_transform(test_x)

In [73]:
from sklearn import linear_model
reg = linear_model.LogisticRegression()
reg.fit(train_x,train_y)
score = reg.score(test_x,test_y)
print('Logistic Regression Accuracy : ',score)

Logistic Regression Accuracy :  0.7425


In [74]:
from sklearn.metrics import classification_report
print(classification_report(test_y,reg.predict(test_x)))

             precision    recall  f1-score   support

          0       0.99      0.74      0.85      1920
          1       0.12      0.85      0.21        80

avg / total       0.96      0.74      0.82      2000



In [75]:
reg2 = linear_model.LogisticRegression(class_weight='balanced')
reg2.fit(train_x,train_y)
score2 = reg2.score(test_x,test_y)
print('Logistic Regression used class weight ' ,score2)
print(classification_report(test_y,reg2.predict(test_x)))

Logistic Regression used class weight  0.7435
             precision    recall  f1-score   support

          0       0.99      0.74      0.85      1920
          1       0.12      0.85      0.21        80

avg / total       0.96      0.74      0.82      2000



In [76]:
# fit model no training data
model = XGBClassifier(learning_rate=0.2,max_depth=7,base_score=0.5,max_delta_step=0,n_estimators=100)

pipe = Pipeline([['xg',MinMaxScaler(feature_range=(0,5))],['model',model]])
pipe.fit(train_x,train_y)
print(classification_report(test_y,pipe.predict(test_x)))
print("ACCURACY:",pipe.score(test_x,test_y))

             precision    recall  f1-score   support

          0       0.98      0.96      0.97      1920
          1       0.35      0.51      0.41        80

avg / total       0.95      0.94      0.95      2000

ACCURACY: 0.942


0.923